# 小売店で扱うデータでデータ加工を行う
## 前提条件
Products A - Z の 26 商品を扱う。
集計期間での商品の単価は変わらない。
売り上げシステムは、従業員が入力し、csvとして保存している。
顧客台帳は、店舗の管理者が週次で集計し、Excel で管理している。
## Data の保存場所
- data_nock/data/ch2/uriage.csv
- 


NotWritableError: The current user does not have write permissions to a required path.
  path: /Applications/JupyterLab.app/Contents/Resources/jlab_server/pkgs/cache/ac2fd263.json
  uid: 501
  gid: 20

If you feel that permissions on this path are set incorrectly, you can manually
change them by executing

  $ sudo chown 501:20 /Applications/JupyterLab.app/Contents/Resources/jlab_server/pkgs/cache/ac2fd263.json

In general, it's not advisable to use 'sudo conda'.



Note: you may need to restart the kernel to use updated packages.


In [2]:
#売り上げデータの読み込み
import pandas as pd 
uriage_data = pd.read_csv("../../data_nock/data/ch2/uriage.csv")
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二


In [3]:
client_data = pd.read_excel("../../data_nock/data/ch2/kokyaku_daicho.xlsx")
client_data.head()

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
client_data = pd.read_excel("../../data_nock/data/ch2/kokyaku_daicho.xlsx")
client_data.head()

In [ ]:
client_data = pd.read_excel("../../data_nock/data/ch2/kokyaku_daicho.xlsx")
client_data.head()